## Задача

В таблице содержится информация о покупках. Необходимо, воспользовавшись этими данными, выяснить, какие пары товаров пользователи чаще всего покупают вместе.

In [2]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori 

In [3]:
purchases = pd.read_csv('https://stepik.org/media/attachments/lesson/409319/test1_completed.csv')
purchases.head()

,id,Товар,Количество
0,17119,Лимон,1.1
1,17119,Лимон оранжевый,0.7
2,17119,Лук-порей,10.0
3,17119,Лук репчатый,2.5
4,17119,Малина свежая,1.0


In [4]:
purchases = purchases.rename(columns={'Товар': 'goods', 'Количество': 'quantity'})

In [5]:
# переводим данные в сводную таблицу, чтобы с ними легче было работать
df = pd.pivot_table(purchases, values='quantity', index='id', columns='goods').fillna(0)

In [6]:
# так как количество товара в данном случае не играет роли, перекодируем купленный товар/не купленный в True/False
def encode_units(x):
    if x <= 0:
        return 0
    else:
        return 1

In [7]:
df_new = df.applymap(encode_units)
df_new.head()

goods,Абрикос вяленый,Абрикосы молдавские,Авокадо ХАСС,Авокадо стандарт,Алыча вяленая,Ананас Gold,Ананасовые кольца,Апельсины столовые,Арбуз,Арбуз овальный,...,Яблоки Гала,Яблоки Голден,Яблоки Джонаголд,Яблоки Мутсу,Яблоки Симиренко,Яблоки Фуджи,Яблоки Чемпион,Яблоки сезонные,Яблоки сушеные,Ягоды Годжи
id,,,,,,,,,,,,,,,,,,,,,
17119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17530,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17618,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
17724,0,1,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
17814,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
transactions = df_new.index.size

In [10]:
## используя модуль apripri ищем наиболее встречаемые пары товаров 
frequent_itemsets = apriori(df_new, min_support=0.05, use_colnames=True)
# так как модуль рассматривает все возможные комбинации, отбираем только те данные, которые для пар товаров
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets = frequent_itemsets.query('length == 2')

frequent_itemsets.head()

,support,itemsets,length
109,0.060800,"(Абрикосы молдавские, Огурцы Луховицкие)",2
110,0.054690,"(Абрикосы молдавские, Укроп)",2
111,0.058356,"(Арбуз, Баклажаны грунтовые)",2
112,0.072411,"(Арбуз, Бананы)",2
113,0.069966,"(Арбуз, Кабачки)",2


In [216]:
# создаем столбик "Встречаемость", как произведение support на количество транзакций
frequent_itemsets['Встречаемость'] = frequent_itemsets['support'] * transactions
frequent_itemsets = frequent_itemsets.drop(columns=['length', 'support'])

In [217]:
frequent_itemsets = frequent_itemsets.sort_values('Встречаемость', ascending=False)

In [218]:
frequent_itemsets = pd.DataFrame(frequent_itemsets).reset_index(drop=True)
frequent_itemsets[['1_Товар', '2_Товар']] = pd.DataFrame(frequent_itemsets.itemsets.to_list())
frequent_itemsets = frequent_itemsets.drop(columns='itemsets')

In [219]:
cols = ['1_Товар', '2_Товар', 'Встречаемость']
frequent_itemsets = frequent_itemsets[cols]

In [222]:
# отбираем топ-5 пар товаров
frequent_itemsets_top = frequent_itemsets.head(5)
frequent_itemsets_top

,1_Товар,2_Товар,Встречаемость
0,Огурцы Луховицкие,Укроп,431.0
1,Петрушка,Укроп,408.0
2,Огурцы Луховицкие,Арбуз,345.0
3,Огурцы Луховицкие,Кабачки,326.0
4,Кинза,Укроп,303.0


# Выводы

* Наиболее часто встречаемые пары товаров состоят преимущественно из овощей, самая распространенная пара это "огурцы и укроп". Вероятно это связано с тем, что это самые распространенные ингредиенты для салата.

* В топ-5 пар попал паттерн "огурцы и арбуз", скорее всего это связано с сезонностью.

* Также стоит отметить, что чаще всего одним из товаров в паре являются "огурцы", что можно объяснить тем, что данный овощ является самым распространенным в продуктовой корзине в принципе.

Подобный анализ продуктовой корзины позволяет оптимизировать размещение товаров в магазине, для увеличения выручки и удобства покупателя, если рассматривать онлайн-магазин, то полученные данные можно использовать для выдачи определенных рекомендаций пользователю в зависимости от продуктов в его корзине.
